In [0]:
from __future__ import print_function
import tensorflow.keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import datetime
import os
import numpy as np
import sklearn
from sklearn.metrics import f1_score, accuracy_score, classification_report
import matplotlib as mpl
#mpl.use('Agg')
import matplotlib.pyplot as plt
import pickle, gzip
import sys
# import sys,shutil,datetime,pickle,codecs,tempfile, gzip

# import numpy as np
# import scipy as sp
import pandas as pd
# import matplotlib as mpl
# import cv2

# import sklearn
# import skimage

# import tensorflow as tf
# import keras
from sklearn.cluster import KMeans, MiniBatchKMeans

In [0]:

def scale(X,testing=False,mode='standard',a=None,b=None):
    #X=np.nan_to_num
    X=np.array(X)
    if  mode=='scale':
        if(not testing):
            mx,mn=X.max(axis=0),X.min(axis=0)
        else:
            mx,mn=b,a
        mx=np.where(mx==mn,mx+1,mx)
        X=(X-mn)/(mx-mn)
        if(testing):return X
        return X,mn,mx
    elif mode=='standard':
        if(not testing):
            mean,std=X.mean(axis=0),X.std(axis=0)
        else:
            mean,std=a,b
        std=np.where(std==0,1,std)
        X=(X-mean)/std
        if(testing):return X
        return X,mean,std

def preprocess(X,model_name,mode='standard',doScale=True,testing=False):
        if(doScale):
            if(not testing):
                X,a,b=scale(X,testing,mode=mode)
                
                if not os.path.isdir(model_name+"_MODEL"):
                   os.makedirs(model_name+"_MODEL")
                np.save('{0}_MODEL/A'.format(model_name),a)
                np.save('{0}_MODEL/B'.format(model_name),b)
                return X
            else:
                a=np.load('{0}_MODEL/A.npy'.format(model_name)).tolist()
                b=np.load('{0}_MODEL/B.npy'.format(model_name)).tolist()
                X=scale(X,testing,'standard',a,b)
                return X

def unpickle(file,is_bytes=True):
  with open(file, 'rb') as fp:
    dict = pickle.load(fp, encoding='bytes')
  return dict

def load_mnist(path,kind):
  'Load MNIST data from `path'
  labels_path = os.path.join(path, '%s-labels-idx1-ubyte.gz' % kind)
  images_path = os.path.join(path, '%s-images-idx3-ubyte.gz' % kind)

  with gzip.open(labels_path, 'rb') as lbpath:
    labels = np.frombuffer(lbpath.read(), dtype=np.uint8, offset=8)

  with gzip.open(images_path, 'rb') as imgpath:
    images = np.frombuffer(imgpath.read(), dtype=np.uint8, offset=16).reshape(len(labels), 784)
  images=pd.DataFrame(images)
  images = np.array( [np.array(x).reshape((28,28,1)) for i,x in images.iterrows()] )
  return images, labels
def load_cifar(data_path = 'data/CIFAR-10'):
  print('path',data_path)
  X,Y,target_names = None, None, None
  for file_name in os.listdir(data_path):
    if 'data_batch' in file_name:
      temp = unpickle( os.path.join(data_path,file_name) )
      X = pd.concat( (pd.DataFrame(temp[b'data']),  X) ) if X is not None else pd.DataFrame( temp[b'data']   )
      Y = pd.concat( (pd.DataFrame(temp[b'labels']),Y) ) if Y is not None else pd.DataFrame( temp[b'labels'] )
    elif 'batches' in file_name:
      temp = unpickle( '/'.join((data_path,file_name)) )
      target_names = temp[b'label_names']
  X = np.array( [np.array(x).reshape((32,32,3)) for i,x in X.iterrows()] )

  #X.to_csv( '/'.join((data_path,'X.csv')), header=False,index=False )
  #Y.to_csv( '/'.join((data_path,'Y.csv')), header=False,index=False )

  #datasets['CIFAR-10'] = (X,Y,[x.decode('ascii') for x in target_names])
  return X,Y

def train_test_split(X,Y,size=0.2):
  #Cross-validation -- to be done via k-fold later.
  from sklearn.model_selection import train_test_split  
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=size)
  return X_train, Y_train,X_test, Y_test

In [0]:

# batch_size = 32
# num_classes = 10
# epochs = 5
# save_dir = os.path.join(os.getcwd(), 'saved_models')
# model_name = 'Fashion-MNIST'
# data_path=os.path.join(os.getcwd(), 'tipr-third-assignment\data')
# if model_name=='CIFAR-10':
#   print('working on CIFAR-10')
#   X,y=load_cifar(data_path+'\\'+model_name)
#   print('x_train shape:', X.shape)
#   print(X.shape[0], 'train samples') 
#   X=preprocess(X,model_name,mode='standard',doScale=True,testing=False)
#   y = tensorflow.keras.utils.to_categorical(y, num_classes)
#   x_train, y_train,x_test, y_test=train_test_split(X,y,size=0.2)
# elif model_name=='Fashion-MNIST':
#   print('working on Fashion-MNIST')
#   X,y=load_mnist(data_path+'\\'+model_name,'train')
#   print('x_train shape:', X.shape)
#   print(X.shape[0], 'train samples') 
#   X=preprocess(X,model_name,mode='standard',doScale=True,testing=False)
#   y = tensorflow.keras.utils.to_categorical(y, num_classes)
#   x_train, y_train,x_test, y_test=train_test_split(X,y,size=0.2)

# # # The data, split between train and test sets:
# # (x_train, y_train), (x_test, y_test) = cifar10.load_data()
# # print('x_train shape:', x_train.shape)
# # print(x_train.shape[0], 'train samples')
# # print(x_test.shape[0], 'test samples')

# # # Convert class vectors to binary class matrices.
# # y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
# # y_test = tensorflow.keras.utils.to_categorical(y_test, num_classes)
# # #x_train,x_test=scale_0_1(x_train,x_test)
# # x_train=preprocess(x_train,model_name,mode='standard',doScale=True,testing=False)
# # x_test=preprocess(x_test,model_name,mode='standard',doScale=True,testing=True)

In [0]:
# #####
# if os.path.isdir(save_dir) and model_name in os.listdir(save_dir):
#     model_path = os.path.join(save_dir, model_name)
#     model2=tensorflow.keras.models.load_model(model_path)
#     print('Loading Saved Model')
# else:
  
#   layers=[512,200]
#   acts=['relu']*len(layers)
#   filters=[4,4]
#   config=[layers]+[acts]+[filters]
#   print('Config:',config)
#   model2=CreateModel(x_train,num_classes,config)
#   print('Creating New One')
# #model=alexnet(x_train,num_classes)


# trainModel(model2,save_dir, model_name,x_train,y_train,x_test,y_test,batch_size,epochs,tbc=None)

In [0]:
def trainModel(model,save_dir, model_name,x_train,y_train,x_test,y_test,batch_size,epochs,tbc,issaveModel=True):
  
  model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            shuffle=True,
            callbacks=[])

  # Save model and weights
  if not os.path.isdir(save_dir):
      os.makedirs(save_dir)
  model_path = os.path.join(save_dir, model_name)
  
  if model_name in os.listdir(save_dir):
    print('Creating New Instance')
    os.remove(model_path)
  if issaveModel:
    model.save(model_path)
    print('Saved trained model at %s ' % model_path)

  # Score trained model.
  scores = model.evaluate(x_test, y_test, verbose=1)
  print('Test loss:', scores[0])
  print('Test accuracy:', scores[1])

In [0]:
def CreateModel(x_train,num_classes, config,mode='glorot_uniform'):
  model = Sequential()
  actor=config[1][0]
  model.add(Conv2D(32, (3, 3), padding='same',
                   input_shape=x_train.shape[1:]))
  model.add(Activation(actor))
  
  #filter
  filters=config[2]
  for filter in filters:
    model.add(Conv2D(32, (filter, filter), padding='same'))
    model.add(Activation(actor))            
    #Pooling
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

  

  model.add(Flatten())
  #DENSE-FULLY CONNECTED LAYERS
  for layer,act in zip(config[0],config[1]):
    model.add(Dense(layer,kernel_initializer=mode))
    model.add(Activation(act))
    model.add(Dropout(0.5))
  model.add(Dense(num_classes,kernel_initializer=mode))
  model.add(Activation('softmax'))

  # initiate RMSprop optimizer
  opt = tensorflow.keras.optimizers.Adam(lr=0.001, decay=1e-6)

  # Let's train the model using RMSprop
  model.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])
  return model

In [0]:
def getF1Scores(aa,bb):
        micron=sklearn.metrics.f1_score(aa,bb,average='micro')
        macron=sklearn.metrics.f1_score(aa,bb,average='macro')
        return micron, macron

In [0]:
# layers=[512,200]
# acts=['relu']*len(layers)
# filters=[4,4]
# config=[layers]+[acts]+[filters]

# model=CreateModel(x_train,num_classes,config)   
# trainModel(model,x_train,y_train,x_test,y_test,batch_size,epochs)


In [0]:
def scale_0_1(x_train,x_test):
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  x_train /= 255
  x_test /= 255
  return x_train,x_test


In [0]:
# %tensorboard --logdir=/content/log/fit
# #!rm -rf ./log/
# log_dir="log/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# tbc = tensorflow.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# #!pip uninstall -q tf-nightly-2.0-preview

# # Load the TensorBoard notebook extension
# from tensorboard import *
# %load_ext tensorboard.notebook

In [0]:
def task1(x_train,y_train,x_test,y_test):  
  batch_size = 32
  num_classes = 10
  epochs = 20
  save_dir = os.path.join(os.getcwd(), 'saved_models')
  model_name = 'Task1'
  
  alayers=[512,128,50]
  total={}
  for i in range(len(alayers)):
    layers=alayers[0:(i+1)]
    acts=['relu']*len(layers)
    filters=[5]*len(layers)
    config=[layers]+[acts]+[filters]
    print('Config:',config)
    model=CreateModel(x_train,num_classes,config)
    print('Creating New One')
    #Hope data is scaled...
    trainModel(model,save_dir, model_name+'layers'+str(i),x_train,y_train,x_test,y_test,batch_size,epochs,tbc=None)
    y_pred=model.predict(x_train)
    y_pred1=1*(y_pred==y_pred.max(axis=1,keepdims=True))
    score=model.evaluate(x_train, y_train, verbose=1)
    y_pred=model.predict(x_test)
    y_pred=1*(y_pred==y_pred.max(axis=1,keepdims=True))
    score=score+[sklearn.metrics.accuracy_score(y_test,y_pred)]+list(getF1Scores(y_train,y_pred1))
    total[i+1]=(score)
    print(score)
  plotGraph(total,model_name,title=model_name)
  return total

In [0]:
# task1(x_train,y_train,x_test,y_test)

In [0]:
def plotGraph(costs,fig_name,Xtitle='Layer Count',title="Some Graph"):
    aa=list(costs.values())   
    aa=np.array([list(i) for i in  aa])
    a1,a2,a3,a4,a5=aa.T #accuracy, cost
   
    plt.figure(num=None, figsize=(4, 4), dpi=300, facecolor='w', edgecolor='k')
    #write after this line.
    plt.ylabel("Accuracy/Cost<Scaled-down by max={0}>".format(int(np.max(a1))))
    plt.xlabel(Xtitle)
    plt.title(title)
    plt.subplot().plot(list(costs.keys()),a2,'*',label='Accuracy on Train Set')
    plt.subplot().plot(list(costs.keys()),a1/np.max(a1),'b', label='Cost of Train Data')
    plt.subplot().plot(list(costs.keys()),a3,'b--', label='Accuracy on Validation Set')
    plt.subplot().plot(list(costs.keys()),a4, label='f1-micro')
    plt.subplot().plot(list(costs.keys()),a5, label='f1-macro')
    
    plt.legend(loc='best', shadow=False)

    plt.savefig(fig_name)
    plt.show()
        

In [0]:
def task2(x_train,y_train,x_test,y_test):  
  batch_size = 1024
  num_classes = 10
  epochs = 10
  save_dir = os.path.join(os.getcwd(), 'saved_models')
  model_name = 'Task2'
  
  alayers=[128,128,128]
  for i in range(len(alayers)):
    layers=alayers[0:i]
    acts=['relu']*(len(layers)+1)
    filters=[5]*(len(layers)+1)
    val=1
    total={}
    while(val<alayers[i]):
      config=[layers+[val]]+[acts]+[filters]
      print('Config:',config)
      model=CreateModel(x_train,num_classes,config)
      print('Creating New One')
      #Hope data is scaled...
      trainModel(model,save_dir, model_name+'layers'+str(i)+'_val_'+str(val),x_train,y_train,x_test,y_test,batch_size,epochs,tbc=None)
      y_pred=model.predict(x_train)
      y_pred1=1*(y_pred==y_pred.max(axis=1,keepdims=True))
      score=model.evaluate(x_train, y_train, verbose=1)
      y_pred=model.predict(x_test)
      y_pred=1*(y_pred==y_pred.max(axis=1,keepdims=True))
      score=score+[sklearn.metrics.accuracy_score(y_test,y_pred)]+list(getF1Scores(y_train,y_pred1))
      total[val]=(score)
      print(score)
      val=val*2
    print("total::::",total)
    plotGraph(total,model_name+'layers'+str(i+1),model_name+'layers'+str(i))
# task2(x_train,y_train,x_test,y_test)

In [0]:
def task3(x_train,y_train,x_test,y_test):  
  batch_size = 512
  num_classes = 10
  epochs = 10
  save_dir = os.path.join(os.getcwd(), 'saved_models')
  model_name = 'Task3'
  
  alayers=[512,128,64]
  actors=['relu','swish','tanh','sigmoid']
  total={}
  for actor in actors:
    layers=alayers
    acts=[actor]*(len(layers))
    filters=[5]*(len(layers))
    
    config=[alayers]+[acts]+[filters]
    print('Config:',config)
    model=CreateModel(x_train,num_classes,config)
    print('Creating New One')
    #Hope data is scaled...
    trainModel(model,save_dir, model_name+actor,x_train,y_train,x_test,y_test,batch_size,epochs,tbc=None,issaveModel=False)
    y_pred=model.predict(x_train)
    y_pred1=1*(y_pred==y_pred.max(axis=1,keepdims=True))
    score=model.evaluate(x_train, y_train, verbose=1)
    y_pred=model.predict(x_test)
    y_pred=1*(y_pred==y_pred.max(axis=1,keepdims=True))
    score=score+[sklearn.metrics.accuracy_score(y_test,y_pred)]+list(getF1Scores(y_train,y_pred1))
    total[actor]=(score)
    print(score)
  plotGraph(total,model_name,'Activations',model_name)
# task3(x_train,y_train,x_test,y_test)

In [0]:
from tensorflow.keras.layers import Activation
from tensorflow.keras import backend as K
from tensorflow.keras.utils import get_custom_objects

def swish2(x):
    return x*K.sigmoid(x)

get_custom_objects().update({'swish': Activation(swish2)})

def addswish(model):
    model.add(Activation(swish2))

In [0]:
def task4(x_train,y_train,x_test,y_test):
  batch_size = 512
  num_classes = 10
  epochs = 10
  save_dir = os.path.join(os.getcwd(), 'saved_models')
  
  model_name='Task4'
  layers=[512,200]
  acts=['relu']*len(layers)
  filters=[4,4]
  
  modes=['glorot_uniform','glorot_normal']
  total={}
  for mode in modes:
    config=[layers]+[acts]+[filters]
    print('Config:',config)
    model=CreateModel(x_train,num_classes,config,mode)
    print('Creating New One')
    
    trainModel(model,save_dir, model_name,x_train,y_train,x_test,y_test,batch_size,epochs,tbc=None,issaveModel=False)
    y_pred=model.predict(x_train)
    y_pred1=1*(y_pred==y_pred.max(axis=1,keepdims=True))
    score=model.evaluate(x_train, y_train, verbose=1)
    y_pred=model.predict(x_test)
    y_pred=1*(y_pred==y_pred.max(axis=1,keepdims=True))
    score=score+[sklearn.metrics.accuracy_score(y_test,y_pred)]+list(getF1Scores(y_train,y_pred1))
    total[mode]=(score)
    print(score)
  
  
  plotGraph(total,model_name,'Weight Inits',model_name)
  
  
# task4(x_train,y_train,x_test,y_test)

In [0]:
def task5_6(X,y):
  
  for i in range(1,6):
    x_train, y_train,x_test, y_test=train_test_split(X,y,size=i *0.1)
    layers=[128,32]
    acts=[activation]*len(layers)
    filters=config
    config=[layers]+[acts]+[filters]
    model=CreateModel(x_train,num_classes,config)   
    trainModel(model,'saved_models', 'task5_'+str(i),x_train,y_train,x_test,y_test,batch_size,epochs,tbc=None)
    lyr=vectorize(model)
    embeds=get_embeds(model,x_test,lyr,batch_size=512)
    clustering(embeds,list(range(10)))
    taskTSNE(embeds,y)
#task5_6(x_train,y_train)          

In [0]:
def task7(X_train,y_train,X_val,y_val):
  
  model = Sequential()
  
  m=X_train.shape[1]
  l=y_train.shape[1]
  layers=[512,200]
  acts=['relu']*len(layers)
  filters=[4,4]
  config=[layers]+[acts]+[filters]
  model.add(Flatten(input_shape=X_train.shape[1:]))
  #model.add(Dense(512, activation=acts[0], input_dim=m))
  #model.add(Dropout(0.5))
  flag=True
  for layer,act in zip(config[0],config[1]):  
      model.add(Dense(layer, activation=act))
      model.add(Dropout(0.5))

  model.add(Dense(l, activation='softmax'))

  opt = tensorflow.keras.optimizers.Adam(lr=0.001, decay=1e-6)
  # Let's train the model using RMSprop
  model.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])

  model.fit(X_train, y_train,
            epochs=500,
            batch_size=1024)
  y_pred = model.predict(X_val)
  #y_pred=oneHot(y_pred,classes)
  #scoreTrain=model.evaluate(X_train, y_train, batch_size=128)
  #scoreVal = model.evaluate(X_val, y_val, batch_size=128)
  #print(scoreTrain,scoreVal)
  y_val=np.argmax(y_val,axis=1)
  y_pred=np.argmax(y_pred,axis=1)
  #print(y_val,y_pred)
  print('Validation Accuracy:',np.mean(y_val==y_pred))
#task7(x_train,y_train,x_test,y_test)

In [0]:

batch_size = 512
num_classes = 10
epochs = 5
#argv="--test-data|D:\\workspace\\tipr\\tipr 3\\tipr-third-assignment\\data\\|--dataset|Fashion-MNIST".split("|")
#argv="--train-data|D:\\workspace\\tipr\\tipr 3\\tipr-third-assignment\\data\\|--test-data|D:\\workspace\\tipr\\tipr 3\\tipr-third-assignment\\data\\|--dataset|CIFAR-10|--filter-config|[8|4|4]|--activation|relu".split("|")
argv = list(sys.argv)
save_dir = os.path.join(os.getcwd(), 'saved_models')
dataSetName = argv[argv.index('--dataset')+1]
test_path = argv[argv.index('--test-data')+1]
#test_path='D:\\workspace\\tipr\\tipr 3\\tipr-third-assignment\\data\\'
model_name = dataSetName
model_path=""

if '--train-data' in argv:
    print('training Model')
    train_path = argv[argv.index('--train-data')+1]
    config   = []#HiddenLayers for now.
    activation=argv[argv.index('--activation')+1]
    for st in argv[argv.index('--filter-config')+1:]:
        st  = st.strip()
        if st.endswith(']'):
            config.append(int(st.strip('[]')))
            break
        else:
            config.append(int(st.strip('[]')))
    data_path=train_path
    if model_name=='CIFAR-10':
      print('working on CIFAR-10')
      X,y=load_cifar(os.path.join(data_path,model_name))
      print('x_train shape:', X.shape)
      print(X.shape[0], 'train samples') 
      X=preprocess(X,'temp_'+model_name,mode='standard',doScale=True,testing=False)
      y = tensorflow.keras.utils.to_categorical(y, num_classes)
      x_train, y_train,x_test, y_test=train_test_split(X,y,size=0.2)
    elif model_name=='Fashion-MNIST':
      print('working on Fashion-MNIST')
      X,y=load_mnist(os.path.join(data_path,model_name),'train')
      print('x_train shape:', X.shape)
      print(X.shape[0], 'train samples') 
      X=preprocess(X,'temp_'+model_name,mode='standard',doScale=True,testing=False)
      y = tensorflow.keras.utils.to_categorical(y, num_classes)
      x_train, y_train,x_test, y_test=train_test_split(X,y,size=0.2)
    
    layers=[512,200]
    acts=[activation]*len(layers)
    filters=config
    config=[layers]+[acts]+[filters]
    model=CreateModel(x_train,num_classes,config)   
    trainModel(model,save_dir, model_name,x_train,y_train,x_test,y_test,batch_size,epochs,tbc=None)

  
        
else:
    
    model_path=os.path.join(os.getcwd(), 'saved_main_models')
    if os.path.isdir(model_path) and model_name in os.listdir(model_path):
        model=tensorflow.keras.models.load_model(os.path.join(model_path,model_name))
        print('Loading Saved Model from',model_path)
    else:
        raise Exception('Model not found!!!',model_path)
if(dataSetName=="Fashion-MNIST"):
    x_test,y_test=load_mnist(os.path.join(test_path,model_name),'test')
elif(dataSetName=="CIFAR-10" ):
    x_test,y_test=load_cifar(os.path.join(test_path,model_name)) 
x_test=preprocess(x_test,model_name,mode='standard',doScale=True,testing=True)
y_test = tensorflow.keras.utils.to_categorical(y_test, num_classes)
  

y_pred=model.predict(x_test)
y_pred=1*(y_pred==y_pred.max(axis=1,keepdims=True))
score=[sklearn.metrics.accuracy_score(y_test,y_pred)]+list(getF1Scores(y_test,y_pred))
print('\n\nTest Accuracy::{0}\nF1-micro::{1}\nF1-macro::{2}'.format(score[0],score[1],score[2]))
    



In [0]:
def vectorize(model,batch_size = 100):
  X, flg, layers_name = None, False, list(map(str,model.layers))[::-1]
  for lyr in range(len(layers_name)):
    if 'Activation' in layers_name[lyr]:
      if flg:
        break
      flg = True
  lyr = -1*(lyr+1)
  print(model.layers)
  return lyr
def get_embeds(model,data,lyr,batch_size=100):
  repr_fun = K.function([model.layers[0].input], [model.layers[lyr].output])
  for i in range(int(np.ceil(len(data)/batch_size))):
    tmp = repr_fun([data[i*batch_size:(i+1)*batch_size]])[0]
    X = pd.concat( (X,pd.DataFrame(tmp)) ) if X is not None else pd.DataFrame(tmp)
  vec_rep = np.array(X)
  return vec_rep
  
def clustering(data,labels):
  km = MiniBatchKMeans(n_clusters=10).fit(data)
  y_true, y_n = np.argmax(labels,axis=1), km.labels_

  l2i = {}
  for i in range(len(data)):
    if y_n[i] not in l2i:
      l2i[ y_n[i] ] = {i}
    else:
      l2i[ y_n[i] ].add(i)

  bestcluster = {}
  for lbl in l2i:
    tmp = {}
    for i in l2i[lbl]:
      if y_true[i] not in tmp:
        tmp[y_true[i]] =  1
      else:
        tmp[y_true[i]] += 1
    bestcluster[lbl] = tmp
  remaining_labels = set(list(range(10)))
  lbl2pred = {}
  TRUE, FALSE = 0, 0
  lambda2=lambda x: bestcluster[lbl][x] if (x in remaining_labels) else 0
  for _ in range(10):
    assgn_lbl = max( l2i,key=lambda lbl:max(bestcluster[lbl], key=lambda2(x)) )
    pred_freq = sorted(bestcluster[assgn_lbl].items(),key=lambda x:x[1],reverse=True)
    for act_lbl, true_freq in pred_freq:
      if act_lbl in remaining_labels:
        TRUE  += true_freq
        FALSE += sum(y for x,y in pred_freq) - true_freq
        lbl2pred[assgn_lbl] = [act_lbl]
        break
    else:
      raise Exception('No remaing Label, should not happen')
  print('Clustering Accuracy is {0:.2f}%'.format(100*TRUE/(TRUE+FALSE)))

def taskTSNE(X,y):
  target_names=[i for i in range(10)]
  plt.legend(bbox_to_anchor=(1.08,0.7), loc="center left", borderaxespad=0)
  plt.axis('off')
  if len(y.shape)>1:
    y2 = np.argmax(y,axis=1)
  else:
    y2 = y
  y2set = set(y2)
  X2 = TSNE(n_iter=250).fit(X)
  for y_n in y2_set:
    x_n = X2[ [i for i in range(X2.shape[0]) if y2[i]==y_n] ].T
    plt.scatter(x_n[0],x_n[1],label=str(target_names[y_n]),s=3)
  plt.savefig('TSNE--{0}--{1:0.2f}.png'.format(name,100*(1-len(X)/len(self.data))),dpi=300,transparent=True,bbox_inches='tight')
  plt.show()